In [3]:
!pip install tensorflow_docs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.6/183.6 kB 1.8 MB/s eta 0:00:00


In [4]:
#===============Import the required libraries===============
import os
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.layers import *
from keras.models import Model
from keras.preprocessing.image import *
from keras.preprocessing.image import load_img, img_to_array
import pandas as pd
from sklearn.metrics import classification_report
#===============Import the required libraries===============

In [5]:
SEED = 999
np.random.seed(SEED)

def load_images_and_labels(image_paths, target_size=(32, 32)):
    images = []
    labels = []

    for image_path in image_paths:
        image = load_img(image_path, target_size=target_size)
        image = img_to_array(image)

        label = image_path.split(os.path.sep)[-2]
        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)


def load_image_paths(drive_folder_path, image_types=('*.jpg', '*.png', '*.jpeg', '*.gif')):
    image_paths = []
    classes = []

    # Get the list of subfolders inside the main folder
    subfolders = [f.path for f in os.scandir(drive_folder_path) if f.is_dir()]

    for folder in subfolders:
        class_name = os.path.basename(folder)  # Get the name of the folder as the class
        class_images = []

        # Iterate over specified image types
        for image_type in image_types:
            class_images.extend(glob(os.path.join(folder, image_type)))

        image_paths.extend(class_images)
        classes.extend([class_name] * len(class_images))

    return image_paths, classes



def build_network(width, height, depth, classes):

    input_layer = Input(shape=(width, height, depth))

    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(input_layer)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    #x = Dropout(rate=0.25)(x)
    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(input_layer)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    #x = Dropout(rate=0.25)(x)
    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(input_layer)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
   # x = Dropout(rate=0.25)(x)
    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(filters=32, kernel_size=(3, 3), padding='same')(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    #x = Dropout(rate=0.25)(x)
    x = Flatten()(x)
    x = Dense(units=512)(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
  #  x = Dropout(rate=0.25)(x)
    x = Dense(units=classes)(x)

    output = Softmax()(x)
    return Model(input_layer, output)


def plot_model_history(model_history, metric, plot_name):
    """
    * plot and save a model's training curve
    """
    sns.set(style='darkgrid')  # Use Seaborn directly to set the style
    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({'Model': model_history}, metric=metric)
    plt.title(f'{metric.upper()}')
    plt.ylim([0, 1])
    plt.savefig(f'{plot_name}.png')
    plt.close()

In [6]:
def main():
    # Load the dataset into memory, normalizing the images and one-hot encoding the labels
    drive_folder_path = '/content/drive/MyDrive/Archaeological_Sites_Classification'  # Update with your actual folder path
    image_paths, classes = load_image_paths(drive_folder_path)
    X, y = load_images_and_labels(image_paths)
    X = X.astype('float') / 255.0
    label_binarizer = LabelBinarizer()
    y = label_binarizer.fit_transform(y)

    # Display class distribution
    class_df = pd.DataFrame({'Class': classes})
    print(class_df['Class'].value_counts())

    (X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.33)
    print(f'Shape of X_train: {X_train.shape}')
    print(f'Shape of X_test: {X_test.shape}')
    print(f'Shape of y_train: {y_train.shape}')
    print(f'Shape of y_test: {y_test.shape}')

    EPOCHS = 50
    BATCH_SIZE = 6

    # Before Augmentation
    model = build_network(32, 32, 3, len(label_binarizer.classes_))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=EPOCHS, batch_size=BATCH_SIZE)

    result = model.evaluate(X_test, y_test)
    print(f'Test accuracy: {result[1]}')
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    print(classification_report(y_true_classes, y_pred_classes, target_names=label_binarizer.classes_))

    plot_model_history(history, 'accuracy', 'normal')

    # After Augmentation
    model = build_network(32, 32, 3, len(label_binarizer.classes_))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    augmenter = ImageDataGenerator(
        horizontal_flip=True, rotation_range=30, width_shift_range=0.1,
        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, fill_mode='nearest'
    )
    train_generator = augmenter.flow(X_train, y_train, BATCH_SIZE)
    hist = model.fit(train_generator, validation_data=(X_test, y_test), epochs=EPOCHS)

    result = model.evaluate(X_test, y_test)
    print(f'Test accuracy: {result[1]}')
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    print(classification_report(y_true_classes, y_pred_classes, target_names=label_binarizer.classes_))
    plot_model_history(hist, 'accuracy', 'augmented')

# run script
if __name__ == "__main__":
    main()

um_qais          811
jarash           703
Wadi_Rum         614
Petra            607
Ajloun           572
Roman_Theater    520
Name: Class, dtype: int64
Shape of X_train: (2564, 32, 32, 3)
Shape of X_test: (1263, 32, 32, 3)
Shape of y_train: (2564, 6)
Shape of y_test: (1263, 6)
Epoch 1/50
428/428 [==============================] - 30s 62ms/step - loss: 1.5841 - accuracy: 0.4758 - val_loss: 2.9276 - val_accuracy: 0.3017
Epoch 2/50
428/428 [==============================] - 28s 66ms/step - loss: 1.1699 - accuracy: 0.5546 - val_loss: 1.2659 - val_accuracy: 0.5859
Epoch 3/50
428/428 [==============================] - 25s 59ms/step - loss: 1.0320 - accuracy: 0.6209 - val_loss: 1.1246 - val_accuracy: 0.6184
Epoch 4/50
428/428 [==============================] - 24s 55ms/step - loss: 0.9334 - accuracy: 0.6490 - val_loss: 1.3057 - val_accuracy: 0.5867
Epoch 5/50
428/428 [==============================] - 25s 58ms/step - loss: 0.8352 - accuracy: 0.6923 - val_loss: 1.1100 - val_accuracy: 0.6247
Ep